In [1]:
import pandas as pd
import numpy as np

import os
import urllib
import json

import datetime
import csv
import time

from io import BytesIO

try:
    import Image
except ImportError:
    from PIL import Image
import pytesseract

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
app_id = '125258161474215'
app_secret = '7340f93f8be2a23a8858e8ae24324947'

access_token = app_id + "|" + app_secret
print(access_token)

125258161474215|7340f93f8be2a23a8858e8ae24324947


In [3]:
posts = pd.read_csv('./2016-10-facebook-fact-check/data/facebook-fact-check.csv')
posts.shape

(2282, 12)

In [4]:
photo_mask = posts['Post Type'].str.contains('photo')
photo_posts = posts[photo_mask]
print(photo_posts.shape)

(207, 12)


In [5]:
photo_posts.head()

,account_id,post_id,Category,Page,Post URL,Date Published,Post Type,Rating,Debate,share_count,reaction_count,comment_count
108,184096565021911,1038670552897837,mainstream,ABC News Politics,https://www.facebook.com/ABCNewsPolitics/posts...,2016-09-23,photo,mostly true,NaN,2.0,12.0,9.0
111,184096565021911,1038758292889063,mainstream,ABC News Politics,https://www.facebook.com/ABCNewsPolitics/posts...,2016-09-23,photo,mostly true,NaN,26.0,178.0,84.0
154,184096565021911,1041489252615967,mainstream,ABC News Politics,https://www.facebook.com/ABCNewsPolitics/posts...,2016-09-26,photo,mostly true,yes,NaN,2.0,0.0
155,184096565021911,1041496015948624,mainstream,ABC News Politics,https://www.facebook.com/ABCNewsPolitics/posts...,2016-09-26,photo,mostly true,yes,20.0,39.0,5.0
158,184096565021911,1041504402614452,mainstream,ABC News Politics,https://www.facebook.com/ABCNewsPolitics/posts...,2016-09-26,photo,mostly true,yes,7.0,30.0,9.0


In [6]:
photo_posts = photo_posts.reset_index(drop=True)
photo_posts.head()

,account_id,post_id,Category,Page,Post URL,Date Published,Post Type,Rating,Debate,share_count,reaction_count,comment_count
0,184096565021911,1038670552897837,mainstream,ABC News Politics,https://www.facebook.com/ABCNewsPolitics/posts...,2016-09-23,photo,mostly true,NaN,2.0,12.0,9.0
1,184096565021911,1038758292889063,mainstream,ABC News Politics,https://www.facebook.com/ABCNewsPolitics/posts...,2016-09-23,photo,mostly true,NaN,26.0,178.0,84.0
2,184096565021911,1041489252615967,mainstream,ABC News Politics,https://www.facebook.com/ABCNewsPolitics/posts...,2016-09-26,photo,mostly true,yes,NaN,2.0,0.0
3,184096565021911,1041496015948624,mainstream,ABC News Politics,https://www.facebook.com/ABCNewsPolitics/posts...,2016-09-26,photo,mostly true,yes,20.0,39.0,5.0
4,184096565021911,1041504402614452,mainstream,ABC News Politics,https://www.facebook.com/ABCNewsPolitics/posts...,2016-09-26,photo,mostly true,yes,7.0,30.0,9.0


In [7]:
photo_posts['Rating'].unique()

array(['mostly true', 'no factual content', 'mostly false',
       'mixture of true and false'], dtype=object)

In [8]:
mostly_true_posts = (photo_posts['Rating'] == 'mostly true')
mostly_true_posts.shape

(207,)

In [9]:
mostly_false_mask = (photo_posts['Rating'] == 'mostly false')
photo_posts[mostly_false_mask].shape

(9, 12)

In [10]:
false_mask = photo_posts['Rating'].str.contains('false')
photo_posts[false_mask]

,account_id,post_id,Category,Page,Post URL,Date Published,Post Type,Rating,Debate,share_count,reaction_count,comment_count
28,135665053303678,551473235056189,right,Eagle Rising,https://www.facebook.com/theEagleisRising/post...,2016-09-19,photo,mostly false,NaN,7577.0,2989.0,146.0
32,135665053303678,551519291718250,right,Eagle Rising,https://www.facebook.com/theEagleisRising/post...,2016-09-19,photo,mixture of true and false,NaN,266.0,318.0,32.0
50,135665053303678,552242831645896,right,Eagle Rising,https://www.facebook.com/theEagleisRising/post...,2016-09-22,photo,mixture of true and false,NaN,593.0,732.0,20.0
52,135665053303678,552571698279676,right,Eagle Rising,https://www.facebook.com/theEagleisRising/post...,2016-09-22,photo,mostly false,NaN,980.0,1509.0,59.0
54,135665053303678,552599694943543,right,Eagle Rising,https://www.facebook.com/theEagleisRising/post...,2016-09-22,photo,mixture of true and false,NaN,17586.0,5662.0,628.0
61,135665053303678,553589488177897,right,Eagle Rising,https://www.facebook.com/theEagleisRising/post...,2016-09-26,photo,mostly false,NaN,5870.0,2856.0,122.0
75,346937065399354,1245894178836967,left,Occupy Democrats,https://www.facebook.com/OccupyDemocrats/posts...,2016-09-19,photo,mixture of true and false,NaN,46108.0,31113.0,2988.0
91,346937065399354,1247976241962094,left,Occupy Democrats,https://www.facebook.com/OccupyDemocrats/posts...,2016-09-22,photo,mostly false,NaN,1.0,17525.0,441.0
96,346937065399354,1248526781907040,left,Occupy Democrats,https://www.facebook.com/OccupyDemocrats/posts...,2016-09-23,photo,mostly false,NaN,22075.0,34622.0,2527.0
112,346937065399354,1252720634820988,left,Occupy Democrats,https://www.facebook.com/OccupyDemocrats/posts...,2016-09-26,photo,mixture of true and false,NaN,12626.0,19499.0,780.0


In [11]:
no_factual_mask = photo_posts['Rating'].str.contains('no factual content')
photo_posts[no_factual_mask]

,account_id,post_id,Category,Page,Post URL,Date Published,Post Type,Rating,Debate,share_count,reaction_count,comment_count
13,146422995398181,1451123788261422,left,Addicting Info,https://www.facebook.com/AddictingInfoOrg/post...,2016-09-22,photo,no factual content,NaN,3532.0,6784.0,318.0
14,146422995398181,1451127358261065,left,Addicting Info,https://www.facebook.com/AddictingInfoOrg/post...,2016-09-22,photo,no factual content,NaN,14529.0,12385.0,666.0
15,146422995398181,1451126608261140,left,Addicting Info,https://www.facebook.com/AddictingInfoOrg/post...,2016-09-23,photo,no factual content,NaN,1.0,4497.0,402.0
17,219367258105115,1282958578412639,mainstream,CNN Politics,https://www.facebook.com/cnnpolitics/posts/128...,2016-09-20,photo,no factual content,NaN,3.0,28.0,61.0
19,219367258105115,1285438601497970,mainstream,CNN Politics,https://www.facebook.com/cnnpolitics/posts/128...,2016-09-22,photo,no factual content,NaN,6.0,47.0,52.0
20,219367258105115,1290297081012122,mainstream,CNN Politics,https://www.facebook.com/cnnpolitics/posts/129...,2016-09-26,photo,no factual content,NaN,83.0,207.0,183.0
21,219367258105115,1290445077663989,mainstream,CNN Politics,https://www.facebook.com/cnnpolitics/posts/129...,2016-09-26,photo,no factual content,NaN,70.0,153.0,116.0
22,219367258105115,1290490284326135,mainstream,CNN Politics,https://www.facebook.com/cnnpolitics/posts/129...,2016-09-26,photo,no factual content,yes,68.0,219.0,166.0
25,135665053303678,551096688427177,right,Eagle Rising,https://www.facebook.com/theEagleisRising/post...,2016-09-19,photo,no factual content,NaN,593.0,639.0,29.0
26,135665053303678,551096721760507,right,Eagle Rising,https://www.facebook.com/theEagleisRising/post...,2016-09-19,photo,no factual content,NaN,89.0,200.0,5.0


In [12]:
false_photo_posts = photo_posts[mostly_false_mask | no_factual_mask]
false_photo_posts

,account_id,post_id,Category,Page,Post URL,Date Published,Post Type,Rating,Debate,share_count,reaction_count,comment_count
13,146422995398181,1451123788261422,left,Addicting Info,https://www.facebook.com/AddictingInfoOrg/post...,2016-09-22,photo,no factual content,NaN,3532.0,6784.0,318.0
14,146422995398181,1451127358261065,left,Addicting Info,https://www.facebook.com/AddictingInfoOrg/post...,2016-09-22,photo,no factual content,NaN,14529.0,12385.0,666.0
15,146422995398181,1451126608261140,left,Addicting Info,https://www.facebook.com/AddictingInfoOrg/post...,2016-09-23,photo,no factual content,NaN,1.0,4497.0,402.0
17,219367258105115,1282958578412639,mainstream,CNN Politics,https://www.facebook.com/cnnpolitics/posts/128...,2016-09-20,photo,no factual content,NaN,3.0,28.0,61.0
19,219367258105115,1285438601497970,mainstream,CNN Politics,https://www.facebook.com/cnnpolitics/posts/128...,2016-09-22,photo,no factual content,NaN,6.0,47.0,52.0
20,219367258105115,1290297081012122,mainstream,CNN Politics,https://www.facebook.com/cnnpolitics/posts/129...,2016-09-26,photo,no factual content,NaN,83.0,207.0,183.0
21,219367258105115,1290445077663989,mainstream,CNN Politics,https://www.facebook.com/cnnpolitics/posts/129...,2016-09-26,photo,no factual content,NaN,70.0,153.0,116.0
22,219367258105115,1290490284326135,mainstream,CNN Politics,https://www.facebook.com/cnnpolitics/posts/129...,2016-09-26,photo,no factual content,yes,68.0,219.0,166.0
25,135665053303678,551096688427177,right,Eagle Rising,https://www.facebook.com/theEagleisRising/post...,2016-09-19,photo,no factual content,NaN,593.0,639.0,29.0
26,135665053303678,551096721760507,right,Eagle Rising,https://www.facebook.com/theEagleisRising/post...,2016-09-19,photo,no factual content,NaN,89.0,200.0,5.0


In [13]:
false_photo_posts[['account_id', 'post_id']].values

array([[  146422995398181,  1451123788261422],
       [  146422995398181,  1451127358261065],
       [  146422995398181,  1451126608261140],
       [  219367258105115,  1282958578412639],
       [  219367258105115,  1285438601497970],
       [  219367258105115,  1290297081012122],
       [  219367258105115,  1290445077663989],
       [  219367258105115,  1290490284326135],
       [  135665053303678,   551096688427177],
       [  135665053303678,   551096721760507],
       [  135665053303678,   551473158389530],
       [  135665053303678,   551473235056189],
       [  135665053303678,   551473311722848],
       [  135665053303678,   551519151718264],
       [  135665053303678,   551519225051590],
       [  135665053303678,   551475151722664],
       [  135665053303678,   551475315055981],
       [  135665053303678,   551830561687123],
       [  135665053303678,   551830681687111],
       [  135665053303678,   551844801685699],
       [  135665053303678,   551844865019026],
       [  135

In [14]:
def request_until_succeed(url):
    req = urllib.request.Request(url)
    success = False
    
    i = 0
    while success is False:
        try: 
            response = urllib.request.urlopen(req)
            if response.getcode() == 200:
                success = True
            return response.read()
        except Exception as e:
            if i == 1:
                success = True
                
            i = i + 1
            print(e)
            time.sleep(5)
            
            print("Error for URL %s: %s" % (url, datetime.datetime.now()))
            return -1



#pytesseract.pytesseract.tesseract_cmd = "/home/anaconda/anaconda3/envs/opencv/bin/tesseract"
pytesseract.pytesseract.tesseract_cmd = "tesseract"

In [15]:
account_id, post_id = photo_posts.loc[0, ['account_id', 'post_id']]
print(account_id, post_id)

184096565021911 1038670552897837


In [16]:
def get_idx_from_posts(index, photo_posts):
    account_id, post_id = photo_posts.loc[index, ['account_id', 'post_id']]
    account_id = str(account_id)
    post_id = str(post_id)
    
    return account_id, post_id

In [17]:
def load_facebook_post(idx, photo_posts, access_token):
    
    account_id, post_id = photo_posts.loc[idx, ['account_id', 'post_id']]
    account_id = str(account_id)
    post_id = str(post_id)
    # construct the URL string
    base = "https://graph.facebook.com/v2.10"
    node = "/" + account_id + "_" + post_id
    parameters = "?fields=id,message,full_picture&access_token=%s" % access_token
    url = base + node + parameters
    
    # retrieve data
    req = request_until_succeed(url)

    if req != -1:
        data = json.loads(request_until_succeed(url))
        print(json.dumps(data, indent=4, sort_keys=True))
    
        image_url = data['full_picture']
    
        #io.imshow(io.imread(data['full_picture']))
        #io.show()
    
        #file = BytesIO(urllib.request.urlopen(image_url).read())
        img_file = "./images/" + '{:03d}'.format(idx) + "_" + account_id + "_" + post_id + ".jpg"
    
        file = open(img_file, 'wb')
        file.write(urllib.request.urlopen(image_url).read())
    
        file.close()
    
        img = Image.open(img_file)
        #plt.figure()
        #plt.imshow(img)

        print('***********************************************')
        ocr_text = pytesseract.image_to_string(img, lang='eng')
        ocr_text = ocr_text.replace('\n', ' ').replace('\r', '')
        #ocr_text.replace('\t', ' ')
        ocr_text = ocr_text.replace('  ', ' ')
        print(ocr_text)
        print('***********************************************')
        img.close()
        #input("Press Enter to continue...")

In [18]:
load_facebook_post(2, photo_posts, access_token)

{
    "full_picture": "https://scontent.xx.fbcdn.net/v/t1.0-9/s720x720/14470487_1041489252615967_2767649267078640383_n.jpg?oh=6c0800d7bb562b5e933cd2b56ee58d00&oe=5A6B991C",
    "id": "184096565021911_1041489252615967",
    "message": "There have already been fireworks between Hillary Clinton and Donald J. Trump as the two candidates face off in the first presidential debate. Their best lines so far: http://abcn.ws/2d5dsiK #Debates"
}
***********************************************
m DONALD, I KNOW l‘YOU LIVE IN YOUR i ?OWN REALITY.
***********************************************


In [19]:
url = 'https://graph.facebook.com/v2.10/346937065399354_1253239891435729?fields=id,message,full_picture&access_token=125258161474215|7340f93f8be2a23a8858e8ae24324947'
#url = 'https://graph.facebook.com/v2.10/346937065399354_1253228411436877?fields=id,message,full_picture&access_token=125258161474215|7340f93f8be2a23a8858e8ae24324947'


req = urllib.request.Request(url)
code = 0
try: 
    response = urllib.request.urlopen(req)
    code = response.getcode()
    if code == 200:
        print(code)
        data = json.loads(response.read())
        print(json.dumps(data, indent=4, sort_keys=True))
except Exception as e:
    #rint(code)
    print(e)
    print("Error for URL %s: %s" % (url, datetime.datetime.now()))

HTTP Error 400: Bad Request
Error for URL https://graph.facebook.com/v2.10/346937065399354_1253239891435729?fields=id,message,full_picture&access_token=125258161474215|7340f93f8be2a23a8858e8ae24324947: 2017-11-11 21:01:44.156408


load_facebook_post(0, '346937065399354', '1253239891435729', access_token)

i = 0
for (account_id, post_id) in false_photo_posts[['account_id', 'post_id']].values:
    print(i)

    load_facebook_post(i, str(account_id), str(post_id), access_token)

    i = i + 1

In [20]:
photo_posts.index.values

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [21]:
for idx in photo_posts.index.values:
    print(idx)
    load_facebook_post(idx, photo_posts, access_token)

0
{
    "full_picture": "https://external.xx.fbcdn.net/safe_image.php?d=AQCxT5lkqdSAGxTb&url=http%3A%2F%2Fa.abcnews.com%2Fimages%2FUS%2FAP_Charlotte_Police_Fatal_Shooting2_ml_160922_4x3t_144.jpg&_nc_hash=AQC4cu43u6BKaQCG",
    "id": "184096565021911_1038670552897837",
    "message": "PHOTOS: Protests in Charlotte, NC after fatal police shooting of Keith Lamont Scott: http://abcn.ws/2dfy8DJ"
}
***********************************************

***********************************************
1
{
    "full_picture": "https://scontent.xx.fbcdn.net/v/t1.0-9/14364737_1038758292889063_2351027085110168736_n.jpg?oh=3d64f9900f1fb605298cb853d9b005c3&oe=5AAA68D2",
    "id": "184096565021911_1038758292889063",
    "message": "JUST IN: Ted Cruz announces in Facebook post he will vote for former rival Donald J. Trump  in November:"
}
***********************************************
Tms eIectIun Is unhke any umer In uur nauun‘s msmry LIKE many umer vmers I have sIruggIea m determme me ngm cuurse macnun m

{
    "full_picture": "https://scontent.xx.fbcdn.net/v/t1.0-9/s720x720/14470487_1041489252615967_2767649267078640383_n.jpg?oh=6c0800d7bb562b5e933cd2b56ee58d00&oe=5A6B991C",
    "id": "184096565021911_1041491152615777",
    "message": "There have already been fireworks between Hillary Clinton and Donald J. Trump as the two candidates face off in the first presidential debate. Their best lines so far: http://abcn.ws/2d5dsiK #Debates"
}
***********************************************
m DONALD, I KNOW l‘YOU LIVE IN YOUR i ?OWN REALITY.
***********************************************
13
{
    "full_picture": "https://scontent.xx.fbcdn.net/v/t1.0-9/14433205_1451123788261422_7417386362002566678_n.jpg?oh=e6066a476199a7717237a38168a6a489&oe=5A66ED04",
    "id": "146422995398181_1451123788261422",
    "message": "LIKE Addicting Info for more great content!"
}
***********************************************
THEPRﬂBlEHI WITH 53 “r H ‘ PEﬂPLEISTHAT THEIR MllllTHS ARE ‘ i ..__.._,.A. m. Addicmglnfo 

***********************************************
MUSLIM COUNTRIES HAVE DONATED OVER $250 MILLION TO MY CAMPAIGN . .l’    DO YOU REALLY THINK I’M GOING TO BLAME RADICAL ISLAM?
***********************************************
29
{
    "full_picture": "https://scontent.xx.fbcdn.net/v/t1.0-9/14344848_551473311722848_7687282526946841772_n.jpg?oh=63cd0373fe77fac4f0aaaefeb1376f01&oe=5A66AE90",
    "id": "135665053303678_551473311722848",
    "message": "Eagle Rising"
}
***********************************************
IT WAS I PIIESSIIIIE BIIIIKEB W'I'Il TIMEBS Mill GEIIWEIIIIJIES WIIIEII 'I'll I'I'   wag x a BIIT no wson Jo nun \ tinnonlﬂnﬁms mm m MW
***********************************************
30
{
    "full_picture": "https://scontent.xx.fbcdn.net/v/t1.0-9/14359120_551519151718264_4057589818709982328_n.jpg?oh=5ba2221fbfac8ffad71f081866a0c858&oe=5A6433DA",
    "id": "135665053303678_551519151718264",
    "message": "www.EagleRising.com"
}
***********************************************
mmey 

{
    "full_picture": "https://scontent.xx.fbcdn.net/v/t1.0-9/14344844_552241984979314_482885750144392263_n.jpg?oh=43eb623ec8c8036f0ba30745efa28de1&oe=5A69A15A",
    "id": "135665053303678_552241984979314",
    "message": "Eagle Rising"
}
***********************************************
Take a good look B.L.M.      _ This is what White ,' Privilege ,7/ looks like
***********************************************
46
{
    "full_picture": "https://scontent.xx.fbcdn.net/v/t1.0-9/14390639_552242051645974_1503398985945739028_n.jpg?oh=9f067c56c70bff29bdfcf406c03b73df&oe=5AA4FAE8",
    "id": "135665053303678_552242051645974",
    "message": "Eagle Rising"
}
***********************************************
WIcKEn mmm-
***********************************************
47
{
    "full_picture": "https://scontent.xx.fbcdn.net/v/t1.0-9/14441066_552242144979298_8501675698085039742_n.jpg?oh=f50e0195d2d6f1c42e5eab739f016169&oe=5AA34600",
    "id": "135665053303678_552242144979298",
    "message": "Eagle Ris

***********************************************
[EARN HOWITUJIIANIJlE A GUN AMI] YOU CAN MAKE’AHY SPACE SAFE
***********************************************
64
{
    "full_picture": "https://scontent.xx.fbcdn.net/v/t1.0-9/14446023_553734261496753_8690051826584488359_n.png?oh=f8217caa7f5d3b16156bb3b8b0348819&oe=5A9A583C",
    "id": "135665053303678_553734261496753",
    "message": "Eagle Rising"
}
***********************************************
This liberal moron is the mayor of and the reason there's no leadership there. Another Democrat!
***********************************************
65
{
    "full_picture": "https://scontent.xx.fbcdn.net/v/t1.0-9/14484637_554123321457847_550218260936636535_n.jpg?oh=3f08a302d6daf0f8e291b868e980960d&oe=5A66ED4D",
    "id": "135665053303678_554123321457847",
    "message": "www.EagleRising.com"
}
***********************************************

***********************************************
66
{
    "full_picture": "https://scontent.xx.fbcdn.net/v/t1.

***********************************************
Pres. Obama just announced that after his presidency, he will focus on combating climate change. After he left office, Pres. Clinton founded his own philantrhopic organization that has helped               Here Is Pres. Carter, age 91, still building homes for the homeless after beating brain cancer. {DEMOCRATS
***********************************************
78
{
    "full_picture": "https://scontent.xx.fbcdn.net/v/t1.0-9/14322320_1245904742169244_3080753052001873871_n.png?oh=3c80c3efdb9dd6e5b98a439ed65f4af4&oe=5A6706B9",
    "id": "346937065399354_1245904742169244",
    "message": "YES!"
}
***********************************************
'Like' if you want Republicans to pay taxpayers back for every single penny they wasted on the " Qenghazi committee I I ‘
***********************************************
79
{
    "full_picture": "https://scontent.xx.fbcdn.net/v/t1.0-9/p720x720/14322356_1245912415501810_1356045226104075981_n.png?oh=b1244f4

1'7". .'r ,- 1. lWhile you were bitching about 10 cents/day of your taxes spent on food stamps... R” The Pentagon squandered $43, 000 per household on the 1 ‘ wars in Iraq and Afghanistan.
***********************************************
92
{
    "full_picture": "https://scontent.xx.fbcdn.net/v/t1.0-9/14440631_1248454695247582_2111550401121076730_n.jpg?oh=8fb116c9660be8950f73d0317c4fe5f4&oe=5AACDFC2",
    "id": "346937065399354_1248454695247582",
    "message": "Great question..."
}
***********************************************
They boo a black man who doesn't stand for the American flag‘ Then they boo a black man who does. Could it be that (he llag isn't the reason khey're booing?
***********************************************
93
{
    "full_picture": "https://scontent.xx.fbcdn.net/v/t1.0-9/s720x720/14440950_1465971186747188_8258637701194297278_n.jpg?oh=8614debeb93aa16c87508f90dbe199d4&oe=5AAD3EDD",
    "id": "346937065399354_1248598775233174",
    "message": "Nailed it."
}
********

URLError: <urlopen error [Errno 65] No route to host>

In [ ]:
def write_csv_file(csvfile, idx, photo_posts, access_token):
        
    account_id, post_id = photo_posts.loc[idx, ['account_id', 'post_id']]
    account_id = str(account_id)
    post_id = str(post_id)
    
    csv_writer = csv.writer(csvfile, delimiter=';',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    # construct the URL string
    base = "https://graph.facebook.com/v2.10"
    node = "/" + account_id + "_" + post_id
    parameters = "?fields=id,message,full_picture&access_token=%s" % access_token
    url = base + node + parameters
    
    req = request_until_succeed(url)

    if req != -1:
        data = json.loads(request_until_succeed(url))
        print(json.dumps(data, indent=4, sort_keys=True))
    
        image_url = data['full_picture']
        img_file = "./images/" + '{:03d}'.format(idx) + "_" + account_id + "_" + post_id + ".jpg"
        img = Image.open(img_file)

        if 'message' not in data:
            message = ''
        else:
            message = data['message']
            
        message = message.replace('\n', ' ').replace('\r', '')
        message = message.replace(';', ' ')
        message = message.replace('  ', ' ')
        
        ocr_text = pytesseract.image_to_string(img, lang='eng')
        ocr_text = ocr_text.replace('\n', ' ').replace('\r', '')
        ocr_text = ocr_text.replace(';', ' ')
        #ocr_text.replace('\t', ' ')
        ocr_text = ocr_text.replace('  ', ' ')
        print(ocr_text)
        
        img.close()

            
        csv_writer.writerow([account_id, post_id, message, image_url, img_file, ocr_text])
        #csv_writer.writerow(['aaa', 'bbb'])
    else:
        csv_writer.writerow([account_id, post_id, 'None', 'None', 'None', 'None'])

csvfile = open('test.csv', 'w', newline='')
write_csv_file(csvfile, i, '146422995398181', '1451123788261422', access_token)
csvfile.close()

csvfile = open('test.csv', 'w', newline='')
write_csv_file(csvfile, '346937065399354', '1253239891435729', access_token)
csvfile.close()

csvfile = open('test.csv', 'w', newline='')
write_csv_file(csvfile, 0, '346937065399354', '1248526781907040', access_token)
csvfile.close()

In [ ]:
csvfile = open('false_photo_posts.csv', 'w')

for idx in false_photo_posts.index.values:
    print(idx)
    write_csv_file(csvfile, idx, photo_posts, access_token)
    
csvfile.close()

In [ ]:
csvfile = open('photo_posts.csv', 'w')
for idx in photo_posts.index.values:
    print(idx)
    write_csv_file(csvfile, idx, photo_posts, access_token)
    
csvfile.close()

In [ ]:
def write_csv_file_with_rating(csvfile, idx, photo_posts, access_token):  
        
    account_id, post_id, rating = photo_posts.loc[idx, ['account_id', 'post_id', 'Rating']]
    account_id = str(account_id)
    post_id = str(post_id)
    
    csv_writer = csv.writer(csvfile, delimiter=';',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    # construct the URL string
    base = "https://graph.facebook.com/v2.10"
    node = "/" + account_id + "_" + post_id
    parameters = "?fields=id,message,full_picture&access_token=%s" % access_token
    url = base + node + parameters
    
    req = request_until_succeed(url)

    if req != -1:
        data = json.loads(request_until_succeed(url))
        print(json.dumps(data, indent=4, sort_keys=True))
    
        image_url = data['full_picture']
        img_file = "./images/" + '{:03d}'.format(idx) + "_" + account_id + "_" + post_id + ".jpg"
        img = Image.open(img_file)

        if 'message' not in data:
            message = ''
        else:
            message = data['message']
            
        message = message.replace('\n', ' ').replace('\r', '')
        message = message.replace(';', ' ')
        message = message.replace('  ', ' ')
        
        ocr_text = pytesseract.image_to_string(img, lang='eng')
        ocr_text = ocr_text.replace('\n', ' ').replace('\r', '')
        ocr_text = ocr_text.replace(';', ' ')
        #ocr_text.replace('\t', ' ')
        ocr_text = ocr_text.replace('  ', ' ')
        print(ocr_text)
        
        img.close()

            
        csv_writer.writerow([account_id, post_id, rating, message, image_url, img_file, ocr_text])
        #csv_writer.writerow(['aaa', 'bbb'])
    else:
        csv_writer.writerow([account_id, post_id, 'None', 'None', 'None', 'None', 'None'])

In [ ]:
csvfile = open('photo_posts_with_ratings.csv', 'w')
for idx in photo_posts.index.values:
    print(idx)
    write_csv_file_with_rating(csvfile, idx, photo_posts, access_token)
    
csvfile.close()

In [ ]:
link_mask = posts['Post Type'].str.contains('link')
link_posts = posts[link_mask]

print(link_posts.shape)

In [ ]:
def gather_text(csvfile, idx, access_token):
        
    account_id, post_id = link_posts.loc[idx, ['account_id', 'post_id']]
    account_id = str(account_id)
    post_id = str(post_id)
    
    csv_writer = csv.writer(csvfile, delimiter=';',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    # construct the URL string
    base = "https://graph.facebook.com/v2.10"
    node = "/" + account_id + "_" + post_id
    parameters = "?fields=id,message,full_picture&access_token=%s" % access_token
    url = base + node + parameters
    
    req = request_until_succeed(url)

    if req != -1:
        data = json.loads(request_until_succeed(url))
        print(json.dumps(data, indent=4, sort_keys=True))

        if 'message' not in data:
            message = ''
        else:
            message = data['message']
            
        message = message.replace('\n', ' ').replace('\r', '')
        message = message.replace(';', ' ')
        message = message.replace('  ', ' ')

            
        csv_writer.writerow([message])
        #csv_writer.writerow(['aaa', 'bbb'])
    else:
        csv_writer.writerow([''])

In [ ]:
csvfile = open('text.csv', 'w')
for idx in link_posts.index.values:
    print(idx)
    gather_text(csvfile, idx, access_token)
    
csvfile.close()

In [ ]:
link_posts.index.values
link_posts = link_posts.reset_index(drop=True)
link_posts